# Project Title
### Data Engineering Capstone Project

#### Project Summary
The purpose of this project is to investigate issues that are related to US immigration such as:
1. The gender distribution of the US immigrants.
2. Visa distribution of the US immigrants.
3. Average age per immigrant.
4. Average temperature per month per city.

We will be using three different datasets for this project.
1. I94 immigration dataset from 2016
2. City temperature dataset from Kaggle
3. US city demographic data from OpenSoft.

The end goal of this project is to design four dimension tables and 1 fact table.
Dimension tables:
1. Cities
2. Immigrants
3. Monthly average city temperature
4. Monthly averate city time

Fact table:
1. Immigration.

Spark will be utilized for ETL tasks and the results will be stored in parquet for downstream analysis.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import os
import glob
import re
from datetime import datetime, timedelta
from pprint import pprint
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col, udf, year, month, avg, round, dayofweek, weekofyear, isnull
from pyspark.sql.types import StringType, IntegerType

In [2]:
# Create Spark session
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11").enableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
The purpose of this project is to perform analysis on issues related to US immigration by using factors of coty monthly average temperature, city demographics and seasonality.

#### Describe and Gather Data 
I will be using these three datasets for this project:
1. **I94 Immigration dataset**: This dataset originated from the US National Tourism and Trade Office. It contains various information regarding international visitors coming to USA. [Dataset link.](https://travel.trade.gov/research/reports/i94/historical/2016.html) 
2. **World temperature dataset**: This dataset originated from Kaggle and contains average weather temperatures by city. [Dataset link.](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data)
3. **US city demographic dataset**: This dataset originated from Opensoft and contains information about the demographics of all US cities such as average age, male and female population. [Dataset link.](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)

In [3]:
# Read immigration data
i94_files = glob.glob("../../data/18-83510-I94-Data-2016/*.sas7bdat")
i94_fname = "../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat"
i94_df = spark.read.format("com.github.saurfang.sas.spark").load(i94_fname)

In [4]:
# Read temperature data
temperature_fname = "../../data2/GlobalLandTemperaturesByCity.csv"
temperature_df = spark.read.format("csv").option("delimiter", ",").option("header", "true").load(temperature_fname)

In [5]:
# Read demographics data
demo_fname = "us-cities-demographics.csv"
demo_df = spark.read.format("csv").option("delimiter", ";").option("header", "true").load(demo_fname)

### Step 2: Explore and Assess the Data
#### Explore the Data

##### I94 Immigration dataset

In [6]:
# Explore immigration data
i94_df.count()

3096313

In [7]:
i94_df.limit(5).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,None,NaN,...,U,None,1979.0,10282016,None,None,None,1.897628e+09,None,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,None,1991.0,D/S,M,None,None,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,None,M,1961.0,09302016,M,None,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,1988.0,09302016,None,None,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,2012.0,09302016,None,None,AA,9.246846e+10,00199,B2


In [8]:
# Create list of valid ports
i94_sas_label_descriptions_fname = "I94_SAS_Labels_Descriptions.SAS"
with open(i94_sas_label_descriptions_fname) as f:
    lines = f.readlines()

re_compiled = re.compile(r"\'(.*)\'.*\'(.*)\'")
valid_ports = {}
for line in lines[302:961]:
    results = re_compiled.search(line)
    valid_ports[results.group(1)] = results.group(2)
print(len(valid_ports))

659


In [9]:
# Create list of valid states
valid_states = demo_df.toPandas()["State Code"].unique()
print(len(valid_states))
print(valid_states)

49
['MD' 'MA' 'AL' 'CA' 'NJ' 'IL' 'AZ' 'MO' 'NC' 'PA' 'KS' 'FL' 'TX' 'VA' 'NV'
 'CO' 'MI' 'CT' 'MN' 'UT' 'AR' 'TN' 'OK' 'WA' 'NY' 'GA' 'NE' 'KY' 'SC' 'LA'
 'NM' 'IA' 'RI' 'PR' 'DC' 'WI' 'OR' 'NH' 'ND' 'DE' 'OH' 'ID' 'IN' 'AK' 'MS'
 'HI' 'SD' 'ME' 'MT']


In [10]:
# Create udf to convert SAS date to PySpark date 
@udf(StringType())
def convert_datetime(x):
    if x:
        return (datetime(1960, 1, 1).date() + timedelta(x)).isoformat()
    return None

In [11]:
# Create udf to validate state
@udf(StringType())
def validate_state(x):  
    if x in valid_states:
        return x
    return 'other'

In [12]:
# Clean immigration data

# Remove any missing values
cleaned_i94_df = i94_df.dropna(how="any", subset=["i94port", "i94addr", "gender"])

# Extract valid states 
cleaned_i94_df = cleaned_i94_df.withColumn("i94addr", validate_state(cleaned_i94_df.i94addr))

# Convert arrival_date (SAS format) to PySpark format
cleaned_i94_df = cleaned_i94_df.withColumn("arrdate", convert_datetime(cleaned_i94_df.arrdate))

# only keep us related immigration data
cleaned_i94_df = cleaned_i94_df.filter(cleaned_i94_df.i94addr != 'other')

staging_i94_df = cleaned_i94_df.select(col("cicid").alias("id"), 
                                       col("arrdate").alias("date"),
                                       col("i94port").alias("city_code"),
                                       col("i94addr").alias("state_code"),
                                       col("i94bir").alias("age"),
                                       col("gender").alias("gender"),
                                       col("i94visa").alias("visa_type"),
                                       "count").drop_duplicates()

staging_i94_df.limit(5).toPandas()

,id,date,city_code,state_code,age,gender,visa_type,count
0,168.0,2016-04-01,WAS,DC,34.0,M,2.0,1.0
1,383.0,2016-04-01,MIA,FL,40.0,M,2.0,1.0
2,608.0,2016-04-01,TOR,TX,45.0,M,1.0,1.0
3,930.0,2016-04-01,NEW,NY,49.0,F,2.0,1.0
4,1229.0,2016-04-01,NYC,CT,32.0,M,1.0,1.0


In [13]:
staging_i94_df.printSchema()

root
 |-- id: double (nullable = true)
 |-- date: string (nullable = true)
 |-- city_code: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- age: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- visa_type: double (nullable = true)
 |-- count: double (nullable = true)



##### Temperature dataset

In [14]:
# Explore temperature dataset
temperature_df.count()

8599212

In [15]:
temperature_df.limit(3).toPandas()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.7369999999999999,Århus,Denmark,57.05N,10.33E
1,1743-12-01,None,None,Århus,Denmark,57.05N,10.33E
2,1744-01-01,None,None,Århus,Denmark,57.05N,10.33E


In [16]:
# Create udf to map city full name to city port abbreviation
@udf(StringType())
def city_to_port(city):
    for key in valid_ports:
        if city.lower() in valid_ports[key].lower():
            return key

In [17]:
# Clean temperature data

# Only use temperatures from United States
# Map full name to city port abbreviation
# Remove invalid ports
cleaned_temp_df = temperature_df.filter(temperature_df["Country"] == "United States") \
    .withColumn("year", year(temperature_df['dt'])) \
    .withColumn("month", month(temperature_df["dt"])) \
    .withColumn("i94port", city_to_port(temperature_df["City"])) \
    .withColumn("AverageTemperature", col("AverageTemperature").cast("float")) \
    .dropna(how='any', subset=["i94port"])

# Only use temperatures from 2013 (the latest year in the dataset)
cleaned_temp_df = cleaned_temp_df.filter(cleaned_temp_df["year"] == 2013)

staging_temp_df = cleaned_temp_df.select(col("year"), col("month"), col("i94port").alias("city_code"),
                                         round(col("AverageTemperature"), 1).alias("avg_temperature"),
                                         col("Latitude").alias("lat"), col("Longitude").alias("long")).drop_duplicates()

print(staging_temp_df.count())
staging_temp_df.limit(5).toPandas()

1044


,year,month,city_code,avg_temperature,lat,long
0,2013,4,COL,16.9,32.95N,85.21W
1,2013,1,DAB,0.5,39.38N,83.24W
2,2013,1,ONT,6.8,34.56N,116.76W
3,2013,2,POM,5.8,45.81N,123.46W
4,2013,5,PRO,14.3,42.59N,72.00W


In [18]:
staging_temp_df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- city_code: string (nullable = true)
 |-- avg_temperature: float (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)



##### Demographics dataset

In [19]:
# Explore demographics dataset
demo_df.count()

2891

In [20]:
demo_df.limit(5).toPandas()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.6,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127,87105,175232,5821,33878,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040,143873,281913,5829,86253,2.73,NJ,White,76402


In [21]:
# Clean demographics dataset
# Calculate percentages of numeric columns and create new ones
cleaned_demo_df = demo_df.withColumn("median_age", demo_df['Median Age']) \
    .withColumn("pct_male_pop", (demo_df['Male Population'] / demo_df['Total Population']) * 100) \
    .withColumn("pct_female_pop", (demo_df['Female Population'] / demo_df['Total Population']) * 100) \
    .withColumn("pct_veterans", (demo_df['Number of Veterans'] / demo_df['Total Population']) * 100) \
    .withColumn("pct_foreign_born", (demo_df['Foreign-born'] / demo_df['Total Population']) * 100) \
    .withColumn("pct_race", (demo_df['Count'] / demo_df['Total Population']) * 100) \
    .withColumn("city_code", city_to_port(demo_df["City"])) \
    .dropna(how='any', subset=["city_code"])

cleaned_demo_df = cleaned_demo_df.select(col("City").alias("city_name"), col("State Code").alias("state_code"), 
                         "median_age", "pct_male_pop", "pct_female_pop","pct_veterans", 
                         "pct_foreign_born", col("Total Population").alias("total_pop"), 
                         col("Race").alias("race"), "pct_race").drop_duplicates()

cleaned_demo_df.count()

883

In [22]:
# Pivot the race column
pivot_demo_df = cleaned_demo_df.groupBy("city_name", "state_code", "median_age", "pct_male_pop",
                                        "pct_female_pop","pct_veterans", "pct_foreign_born", "total_pop").pivot("Race").avg("pct_race")

pivot_demo_df = pivot_demo_df.withColumn("city_code", city_to_port(pivot_demo_df["city_name"])) \
    .dropna(how='any', subset=["city_code"])

staging_demo_df = pivot_demo_df.select("city_code", "state_code", "city_name", "median_age",
                                    round(col("pct_male_pop"), 1).alias("pct_male_pop"),
                                    round(col("pct_female_pop"), 1).alias("pct_female_pop"),
                                    round(col("pct_veterans"), 1).alias("pct_veterans"),
                                    round(col("pct_veterans"), 1).alias("pct_foreign_born"),
                                    round(col("American Indian and Alaska Native"), 1).alias("pct_native_american"),
                                    round(col("Asian"), 1).alias("pct_asian"),
                                    round(col("Black or African-American"), 1).alias("pct_black"),
                                    round(col("Hispanic or Latino"), 1).alias("pct_hispanic_or_latino"),
                                    round(col("White"), 1).alias("pct_white"), "total_pop")
print(staging_demo_df.count())
staging_demo_df.limit(10).toPandas()

180


,city_code,state_code,city_name,median_age,pct_male_pop,pct_female_pop,pct_veterans,pct_foreign_born,pct_native_american,pct_asian,pct_black,pct_hispanic_or_latino,pct_white,total_pop
0,TUC,AZ,Tucson,33.6,49.8,50.2,7.2,7.2,4.6,4.6,6.4,43.5,76.1,531674
1,MCA,TX,Allen,37.2,52.3,47.7,3.6,3.6,0.2,16.1,13.4,10.8,71.2,98138
2,CRP,TX,Corpus Christi,35.0,49.5,50.5,7.7,7.7,0.9,2.8,4.6,61.9,90.3,324082
3,FMY,FL,Fort Myers,37.3,49.8,50.2,5.8,5.8,NaN,4.8,23.4,24.1,67.8,74015
4,ORL,FL,Orlando,33.1,48.3,51.7,4.7,4.7,0.9,4.1,25.1,33.0,66.1,270917
5,LOS,CA,Los Angeles,35.0,49.3,50.7,2.2,2.2,1.6,12.9,10.2,48.8,54.8,3971896
6,PRO,RI,Providence,29.9,49.7,50.3,2.8,2.8,2.3,7.5,17.1,43.5,54.6,179204
7,CID,IA,Cedar Rapids,36.2,48.4,51.6,6.0,6.0,1.0,4.1,9.1,4.1,89.6,130405
8,SPI,IL,Springfield,38.8,47.2,52.8,6.4,6.4,1.4,3.3,21.5,2.3,77.2,117809
9,POM,OR,Portland,36.7,49.6,50.4,4.7,4.7,2.4,10.2,7.3,9.7,82.9,632187


In [23]:
staging_demo_df.printSchema()

root
 |-- city_code: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- city_name: string (nullable = true)
 |-- median_age: string (nullable = true)
 |-- pct_male_pop: double (nullable = true)
 |-- pct_female_pop: double (nullable = true)
 |-- pct_veterans: double (nullable = true)
 |-- pct_foreign_born: double (nullable = true)
 |-- pct_native_american: double (nullable = true)
 |-- pct_asian: double (nullable = true)
 |-- pct_black: double (nullable = true)
 |-- pct_hispanic_or_latino: double (nullable = true)
 |-- pct_white: double (nullable = true)
 |-- total_pop: string (nullable = true)



### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
We have chosen star schema for the data model because it is simple to be utilized. Analysis can be done easily by joining fact and dimension tables.

![Data Model](data_model.png)

These are the details for the data schema:

**Staging Tables**

- staging_i94_df
    - id
    - date
    - city_code
    - state_code
    - age
    - gender
    - visa_type
    - count

- staging_temp_df
    - year
    - month
    - city_code
    - city_name
    - avg_temperature
    - lat
    - long

- staging_demo_df
    - city_code
    - state_code
    - city_name
    - median_age
    - pct_male_pop
    - pct_female_pop
    - pct_veterans
    - pct_foreign_born
    - pct_native_american
    - pct_asian
    - pct_black
    - pct_hispanic_or_latino
    - pct_white
    - total_pop
 
**Fact Table**
- immigration_df
    - id
    - state_code
    - city_code
    - date
    - count
 
**Dimension Tables**
- immigrant_df
    - id
    - gender
    - age
    - visa_type

- city_df
    - city_code
    - state_code
    - city_name
    - median_age
    - pct_male_pop
    - pct_female_pop
    - pct_veterans
    - pct_foreign_born
    - pct_native_american
    - pct_asian
    - pct_black
    - pct_hispanic_or_latino
    - pct_white
    - total_pop
    - lat
    - long

- monthly_city_temp_df
    - city_code
    - year
    - month
    - avg_temperature

- time_df
    - date
    - dayofweek
    - weekofyear
    - month

#### 3.2 Mapping Out Data Pipelines
We have taken these steps to create the data pipelines:
1. Clean the datasets based on nulls, data types, duplicates and etc.
2. Load the taging tables (staging_i94_df, staging_tempt_def and staging_demo_df)
3. Create dimension tables ( immigrant_df, city_df, monthly_city_temp_df and time_df)
4. Create a fact table (immigration_df) with information on immigration count, mapping id in immigrant_df, city_code in city_df and monthly_city_temp_df and date in time_df ensuring referential integrity.
5. Save the fact and dimension tables via parquet for downstream query.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [24]:
# Create dimension table for immigrant
immigrant_df = staging_i94_df.select("id", "gender", "age", "visa_type").drop_duplicates()

In [25]:
immigrant_df.count()

2435922

In [26]:
immigrant_df.limit(5).toPandas()

,id,gender,age,visa_type
0,270799.0,F,49.0,2.0
1,275758.0,M,50.0,2.0
2,445416.0,M,69.0,1.0
3,488015.0,M,6.0,2.0
4,503609.0,M,21.0,2.0


In [27]:
# Create dimension table for city
city_df = staging_demo_df.join(staging_temp_df, "city_code") \
    .select("city_code", "state_code", "city_name", "median_age", "pct_male_pop", "pct_female_pop", "pct_veterans",
           "pct_foreign_born", "pct_native_american", "pct_asian", "pct_black",
           "pct_hispanic_or_latino", "pct_white", "total_pop", "lat", "long").drop_duplicates()

In [28]:
city_df.count()

142

In [29]:
city_df.limit(5).toPandas()

,city_code,state_code,city_name,median_age,pct_male_pop,pct_female_pop,pct_veterans,pct_foreign_born,pct_native_american,pct_asian,pct_black,pct_hispanic_or_latino,pct_white,total_pop,lat,long
0,BRO,TX,Brownsville,30.6,47.7,52.3,2.3,2.3,0.6,0.9,0.7,92.5,95.0,183888,26.52N,96.72W
1,HSV,WI,Madison,30.7,49.2,50.8,3.9,3.9,0.9,9.6,8.2,7.9,82.1,248956,34.56N,85.62W
2,ATL,GA,Atlanta,33.8,48.3,51.7,4.0,4.0,1.0,5.2,52.9,4.0,42.3,463875,34.56N,83.68W
3,NEW,NJ,Newark,34.6,49.0,51.0,2.1,2.1,0.8,2.6,51.4,35.6,27.1,281913,40.99N,74.56W
4,NWH,CT,New Haven,29.9,48.9,51.1,2.0,2.0,1.7,6.1,33.3,33.4,43.2,130310,40.99N,72.43W


In [30]:
# Create dimension table for monthly city temperature
monthly_city_temp_df = staging_temp_df.select("city_code", "year", "month", "avg_temperature").drop_duplicates()

In [31]:
monthly_city_temp_df.count()

1043

In [32]:
monthly_city_temp_df.limit(5).toPandas()

,city_code,year,month,avg_temperature
0,SAA,2013,6,18.6
1,PHI,2013,5,16.6
2,BOS,2013,5,14.3
3,BUR,2013,3,14.5
4,RNO,2013,2,4.7


In [33]:
# Create dimension table for time
time_df = staging_i94_df.withColumn("dayofweek", dayofweek("date"))\
                .withColumn("weekofyear", weekofyear("date"))\
                .withColumn("month", month("date"))
                        
time_df = time_df.select("date", "dayofweek", "weekofyear", "month").drop_duplicates()

In [34]:
time_df.count()

30

In [35]:
time_df.limit(5).toPandas()

,date,dayofweek,weekofyear,month
0,2016-04-23,7,16,4
1,2016-04-22,6,16,4
2,2016-04-08,6,14,4
3,2016-04-09,7,14,4
4,2016-04-26,3,17,4


In [36]:
# Create fact table for immigration
immigration_df = staging_i94_df.select("id", "state_code", "city_code", "date", "count").drop_duplicates()

In [37]:
immigration_df.count()

2435922

In [38]:
immigration_df.limit(5).toPandas()

,id,state_code,city_code,date,count
0,25716.0,CA,LOS,2016-04-01,1.0
1,56083.0,HI,HHW,2016-04-01,1.0
2,261977.0,FL,NYC,2016-04-02,1.0
3,290139.0,NY,NYC,2016-04-02,1.0
4,487570.0,HI,HHW,2016-04-03,1.0


In [39]:
# Create fact table for immigration
immigration_df = staging_i94_df.select("id", "state_code", "city_code", "date", "count").drop_duplicates()

In [40]:
immigration_df.count()

2435922

In [41]:
immigration_df.limit(5).toPandas()

,id,state_code,city_code,date,count
0,25716.0,CA,LOS,2016-04-01,1.0
1,56083.0,HI,HHW,2016-04-01,1.0
2,261977.0,FL,NYC,2016-04-02,1.0
3,290139.0,NY,NYC,2016-04-02,1.0
4,487570.0,HI,HHW,2016-04-03,1.0


In [ ]:
# Write dimension tables to parquet
immigrant_df.write.mode("overwrite").partitionBy("gender", "age").parquet("immigrants")
city_df.write.mode("overwrite").partitionBy("state_code").parquet("cities")
monthly_city_temp_df.write.mode("overwrite").parquet("monthly_city_temperatues")
time_df.write.mode("overwrite").parquet("time")

# Write fact table to parquet
immigration_df.write.mode("overwrite").partitionBy("state_code", "city_code").parquet("immigration")

#### 4.2 Data Quality Checks
Run Quality Checks

In [ ]:
# Perform quality checks
# Check if the table exists
def table_exists(df):
    if df is not None:
        return True
    else:
        return False
        
if table_exists(immigrant_df) & table_exists(city_df) & table_exists(monthly_city_temp_df) & table_exists(time_df) & table_exists(immigration_df):
    print("data quality check passed")
    print("dimension tables and fact table exist")
    print()
else:
    print("data quality check failed")
    print("table missing...")

In [ ]:
# Check if the table is not empty
def table_not_empty(df):
    return df.count() != 0 

if table_not_empty(immigrant_df) & table_not_empty(city_df) & table_not_empty(monthly_city_temp_df) & table_not_empty(time_df) & table_not_empty(immigration_df):
    print("data quality check passed!")
    print("dimension tables and fact table contain records")
    print()
else:
    print("data quality check failed!")
    print("null records...")

#### 4.3 Data dictionary 
**Fact Table**
- immigration_df
    - id: id
    - state_code: state code of arrival city
    - city_code: city port code of arrival city
    - date: date of arrival
    - count: count of immigrant's entries into the US

**Dimension Tables**
- immigrant_df
    - id: id of immigrant
    - gender: gender of immigrant
    - age: age of immigrant
    - visa_type: immigrant's visa type

- city_df
    - city_code: city port code
    - state_code: state code of the city
    - city_name: name of the city
    - median_age: median age of the city
    - pct_male_pop: city's male population in percentage
    - pct_female_pop: city's female population in percentage
    - pct_veterans: city's veteran population in percentage
    - pct_foreign_born: city's foreign born population in percentage
    - pct_native_american: city's native american population in percentage
    - pct_asian: city's asian population in percentage
    - pct_black: city's black population in percentage
    - pct_hispanic_or_latino: city's hispanic or latino population in percentage
    - pct_white: city's white population in percentage
    - total_pop: city's total population
    - lat: latitude of the city
    - long: longitude of the city

- monthly_city_temp_df
    - city_code: city port code
    - year: year
    - month: month 
    - avg_temperature: average temperature in city for given month

- time_df
    - date: date
    - dayofweek: day of the week
    - weekofyear: week of year
    - month: month

#### Step 5: Complete Project Write Up
1. Clearly state the rationale for the choice of tools and technologies for the project.
    - For this project, I have used Spark since it can easily handle huge datasets that come in multiple file formats. Spark SQL was used to process the input files into dataframes and manipulated via standard SQL join operations to create the tables.

2. Propose how often the data should be updated and why.
    - The data update cycle is typically chosen on two criteria. One is the reporting cycle, the other is the availabilty of new data to be fed into the system. For example, if new batch of average temperature can be made available at monthly interval, we might settle for monthly data refreshing cycle.

Write a description of how you would approach the problem differently under the following scenarios:
1. The data was increased by 100x.
    - Utilize Amazon Redshift service: It is an analytical database that is optimized for aggregation and read-heavy workloads.
2. The data populates a dashboard that must be updated on a daily basis by 7am every day.
    - Utilize Airflow. We can create DAG retries and emails for the purpose of failure notifications.
3. The database needed to be accessed by 100+ people.
    - Utilize Redshift. It has auto-scaling capabilities and good read performance, with larger capacity to serve more users, and workload management to ensure equitable usage of resources across users.